In [1]:
# --- 0. Setup ---
import pandas as pd
import numpy as np
import geopandas as gpd
from collections import Counter
import os

# --- 1. Create 'data' folder ---
os.makedirs('data', exist_ok=True)

# --- 2. Load your original dataset ---
file_path = "data/Food_Inspections.csv"  # <-- Update if needed
df = pd.read_csv(file_path)

print(f"Rows Loaded: {len(df)}")

# --- 3. Initial Cleaning ---
df["Inspection Date"] = pd.to_datetime(df["Inspection Date"], errors="coerce")
df = df[df["Inspection Date"].dt.year >= 2019]
df["Failed"] = df["Results"].str.upper().str.contains("FAIL").astype(int)
df["Month"] = df["Inspection Date"].dt.to_period("M").astype(str)
df["Month"] = pd.to_datetime(df["Month"])
df["Zip"] = df["Zip"].astype(str)
df["Facility Type"] = df["Facility Type"].fillna("UNKNOWN").str.upper().str.strip()
df["Risk"] = df["Risk"].fillna("Not Specified")
df["Risk"] = df["Risk"].str.extract(r'(Risk \d)').fillna("Not Specified")

# --- 4. Create each visualization-specific small CSVs ---

# 4.1 Monthly Failure Rates
monthly_failure = df.groupby("Month")["Failed"].mean().reset_index()
monthly_failure["Smoothed"] = monthly_failure["Failed"].rolling(window=3, center=True).mean()
monthly_failure.to_csv("data/monthly_failure.csv", index=False)

# 4.2 Top 30 ZIP Codes by Failure Rate
failure_by_zip = df.groupby("Zip")["Failed"].mean().reset_index()
failure_by_zip = failure_by_zip.sort_values("Failed", ascending=False).head(30)
failure_by_zip.to_csv("data/top30_zip_failure.csv", index=False)

# 4.3 Top 20 Most Common Violation Codes
df["Violation Codes"] = df["Violations"].str.findall(r'(\d+)\.')
all_codes = [code for codes in df["Violation Codes"].dropna() for code in codes]
code_counts = pd.DataFrame(Counter(all_codes).items(), columns=["Violation Code", "Count"])
code_counts = code_counts.sort_values("Count", ascending=False).head(20)
code_counts.to_csv("data/violation_codes_top20.csv", index=False)

# 4.4 Facility Type vs Inspection Results
facility_result_counts = df.groupby(["Facility Type", "Results"]).size().reset_index(name="Count")
facility_result_counts.to_csv("data/facility_type_results.csv", index=False)

# 4.5 Monthly Inspection Volume
monthly_counts = df.groupby("Month").size().reset_index(name="Inspection Count")
monthly_counts.to_csv("data/monthly_inspection_volume.csv", index=False)

# 4.6 Risk Level - Inspection Outcomes
risk_df = df[df["Risk"].isin(["Risk 1", "Risk 2", "Risk 3"])].copy()
risk_df.to_csv("data/risk_level_failure.csv", index=False)

# 4.7 Time Series for Brushing (Date vs Failed)
df_time_series = df[["Inspection Date", "Failed"]].dropna()
df_time_series.to_csv("data/time_series_failure.csv", index=False)

print("Data preprocessing complete! Small files saved to /data/")



Rows Loaded: 286741
Data preprocessing complete! Small files saved to /data/


In [2]:
# --- New Aggregation: Inspections per Month + Facility Type ---
monthly_facility_counts = (
    df.groupby(["Month", "Facility Type"])
    .size()
    .reset_index(name="Inspection Count")
)

# Save it
monthly_facility_counts.to_csv("data/monthly_facility_counts.csv", index=False)

print("Saved monthly_facility_counts.csv!")


Saved monthly_facility_counts.csv!


In [3]:
# --- Step 1: Make a copy ---
df_copy = df.copy()

# --- Step 2: Filter Risk levels only ---
df_copy = df_copy[df_copy["Risk"].isin(["Risk 1", "Risk 2", "Risk 3"])]

# --- Step 3: Explode Violations ---
df_copy["Violation Codes"] = df_copy["Violations"].str.findall(r'(\d+)\.')
df_exploded = df_copy.explode("Violation Codes")
df_exploded["Violation Codes"] = df_exploded["Violation Codes"].astype(str)

# --- Step 4: Group by Risk + Violation Code ---
violation_by_risk = (
    df_exploded.groupby(["Risk", "Violation Codes"])
    .size()
    .reset_index(name="Count")
)

# --- Step 5: Save the new file ---
violation_by_risk.to_csv('data/violation_by_risk.csv', index=False)


In [4]:
# --- 1. For Facility and Risk Views ---
df_facility = df.copy()
df_facility["Facility Type"] = df_facility["Facility Type"].fillna("UNKNOWN").str.upper().str.strip()

# Save inspection_data.csv
df_facility_small = df_facility[["Inspection Date", "Facility Type", "Failed", "Risk", "Violations"]]
df_facility_small.to_csv("data/inspection_data.csv", index=False)



In [5]:
# Parse dates
df["Inspection Date"] = pd.to_datetime(df["Inspection Date"], errors="coerce")

# Extract year
df["Inspection Year"] = df["Inspection Date"].dt.year

# Filter years 2019–2024
df = df[df["Inspection Year"].isin([2019, 2020, 2021, 2022, 2023, 2024])]

# Create Failed column
df["Failed"] = df["Results"].str.upper().str.contains("FAIL").astype(int)

# Clean Zip
df["Zip"] = df["Zip"].fillna("").astype(str).str.strip()
df["Zip"] = df["Zip"].str.extract(r'(\d{5})')

# Drop NaN Zips
df = df[df["Zip"].notna()]

# Final minimal dataset
df_zip_small = df[["Inspection Date", "Zip", "Failed"]]

# Save
df_zip_small.to_csv('data/inspection_data_for_zip.csv', index=False)

print(f"Final clean inspection_data_for_zip.csv saved with {len(df_zip_small)} rows!")

Final clean inspection_data_for_zip.csv saved with 104461 rows!


In [6]:
# === Step 1: Load ZIP Code GeoJSON ===
geo_path = "data/Boundaries - ZIP Codes_20250402.geojson"
geo = gpd.read_file(geo_path)

# Ensure ZIP code is a zero-padded string
geo["zip"] = geo["zip"].astype(str).str.zfill(5)

# Keep only needed columns
geo = geo[["zip", "geometry"]]

# === Step 2: Load Food Inspection CSV ===
df = pd.read_csv("data/Food_Inspections.csv")

# Parse dates and filter to 2019+
df["Inspection Date"] = pd.to_datetime(df["Inspection Date"], errors="coerce")
df = df[df["Inspection Date"].dt.year >= 2019]

# Clean up result column
df["Results"] = df["Results"].str.upper().str.strip()
df["Failed"] = df["Results"].str.contains("FAIL", na=False).astype(int)

# Normalize ZIP column
df["Zip"] = df["Zip"].astype(str).str.split('.').str[0].str.zfill(5)

# === Step 3: Calculate failure rate per ZIP ===
fail_rate = df.groupby("Zip")["Failed"].mean().reset_index()
fail_rate.columns = ["zip", "FailureRate"]




In [7]:
# Reload dataset after kernel reset
df = pd.read_csv("data/Food_Inspections.csv")

# Convert 'Inspection Date' to datetime
df['Inspection Date'] = pd.to_datetime(df['Inspection Date'], errors='coerce')

# Filter only valid risk levels
valid_risks = ['Risk 1 (High)', 'Risk 2 (Medium)', 'Risk 3 (Low)']
df = df[df['Risk'].isin(valid_risks)].copy()

# Create 'Month' as quarter-based period
df['Month'] = df['Inspection Date'].dt.to_period('Q').astype(str)

# Optional: enforce ordered categories for Risk to fix Y-axis order in heatmap
df['Risk'] = pd.Categorical(df['Risk'], categories=valid_risks, ordered=True)

# build heatmap data
heatmap_data = (
    df.groupby(['Month', 'Risk'], observed=True)
    .agg(
        total_inspections=('Results', 'count'),
        failed_inspections=('Results', lambda x: (x == 'Fail').sum())
    )
    .reset_index()
)

heatmap_data['Failure Rate'] = (
    heatmap_data['failed_inspections'] / heatmap_data['total_inspections'] * 100
).round(2)

# Save cleaned heatmap data
heatmap_data.to_csv("data/heatmap_data.csv", index=False)


# -----------------------------
# 2. Violin Plot Data: Add Month + Risk + Violation Count
# -----------------------------
# Create 'Violation Count' from Violations column
df['Violation Count'] = df['Violations'].fillna('').apply(lambda x: len(x.split('|')) if x else 0)

# Keep only rows with non-null Inspection Type
df_violin = df[['Inspection Type', 'Facility Type', 'Violation Count', 'Month']].dropna()

# Save the new violin dataset
df_violin.to_csv("data/violin_data.csv", index=False)


In [8]:
# Load the original Food Inspections dataset
df = pd.read_csv("data/Food_Inspections.csv")

# Ensure 'Inspection Date' is in datetime format
df['Inspection Date'] = pd.to_datetime(df['Inspection Date'], errors='coerce')

# Keep only valid Risk levels
df = df[df['Risk'].isin(['Risk 1 (High)', 'Risk 2 (Medium)', 'Risk 3 (Low)'])].copy()

# Add 'Month' column as Year-Quarter string
df['Month'] = df['Inspection Date'].dt.to_period('Q').astype(str)

# Create grouped summary for treemap
treemap_data = (
    df.groupby(['Month', 'Risk', 'Facility Type'])
    .agg(
        total_inspections=('Results', 'count'),
        failed_inspections=('Results', lambda x: (x == 'Fail').sum())
    )
    .reset_index()
)

# Add new column with custom name to avoid overwriting
treemap_data["Treemap Failure Rate"] = (
    treemap_data["failed_inspections"] / treemap_data["total_inspections"] * 100
).round(2)

# Save to a new file
treemap_data.to_csv("data/treemap_data_custom.csv", index=False)




In [10]:
# === Step 1: Load ZIP Code GeoJSON ===
geo_path = "data/Boundaries - ZIP Codes_20250402.geojson"
geo = gpd.read_file(geo_path)

# Ensure ZIP code is a zero-padded string
geo["zip"] = geo["zip"].astype(str).str.zfill(5)

# Keep only needed columns
geo = geo[["zip", "geometry"]]

# === Step 2: Load Food Inspection CSV ===
df = pd.read_csv("data/Food_Inspections.csv")

# Parse dates and filter to 2019+
df["Inspection Date"] = pd.to_datetime(df["Inspection Date"], errors="coerce")
df = df[df["Inspection Date"].dt.year >= 2019]

# Clean up result column
df["Results"] = df["Results"].str.upper().str.strip()
df["Failed"] = df["Results"].str.contains("FAIL", na=False).astype(int)

# Normalize ZIP column
df["Zip"] = df["Zip"].astype(str).str.split('.').str[0].str.zfill(5)

# === Step 3: Calculate failure rate per ZIP ===
fail_rate = df.groupby("Zip")["Failed"].mean().reset_index()
fail_rate.columns = ["zip", "FailureRate"]

# === Step 4: Merge with GeoJSON ===
merged = geo.merge(fail_rate, on="zip", how="left")

# === Step 5: Reproject to EPSG:4326 (Web Mercator) ===
if merged.crs is None or merged.crs.to_string() != 'EPSG:4326':
    merged = merged.to_crs(epsg=4326)

# === Step 6: Export minimal GeoJSON ===
merged[["zip", "geometry", "FailureRate"]].to_file("data/minimized_choropleth.json", driver="GeoJSON")

print("✅ Exported 'minimized_choropleth.json' with essential fields.")

✅ Exported 'minimized_choropleth.json' with essential fields.


In [11]:
# Load ZIP GeoJSON
zip_gdf = gpd.read_file("data/Boundaries - ZIP Codes_20250402.geojson")

# ✅ Dissolve all shapes into one — ignore 'zip', 'objectid', etc.
city_boundary = zip_gdf.dissolve()

# ✅ Reset geometry CRS to EPSG:4326 for web maps
city_boundary = city_boundary.to_crs(epsg=4326)

# ✅ Export the true boundary (will now be 1 shape, no zip)
city_boundary.to_file("data/chicago_boundary.json", driver="GeoJSON")

print("✅ Exported simplified chicago_boundary.json")


✅ Exported simplified chicago_boundary.json


In [24]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import re

# Clean column names
df.columns = df.columns.str.strip()

# Use only required columns and drop rows with missing data
df_clean = df[['Facility Type', 'Results', 'Violations']].dropna()

# Map results to numeric scores (proxy for inspection score)
result_map = {
    'Pass': 2,
    'Pass w/ Conditions': 1,
    'Fail': 0
}
df_clean['Inspection Score'] = df_clean['Results'].map(result_map)

# Drop rows with unmapped results (like 'Out of Business')
df_clean = df_clean.dropna(subset=['Inspection Score'])

# Improved violation counter using regex (counts patterns like "10. " or "5. ")
def count_violations(v):
    if isinstance(v, str):
        return len(re.findall(r'\d+\.', v))
    return 1

df_clean['Violation Count'] = df_clean['Violations'].apply(count_violations)


# ✅ Violation severity scoring
severity_keywords = {
    'rodent': 3,
    'pest': 2,
    'sewage': 4,
    'contamination': 3,
    'temperature': 2,
    'cleaning': 1,
    'food': 1,
    'hygiene': 2,
    'handwashing': 2
}

def compute_severity(violation_text):
    text = str(violation_text).lower()
    return sum(weight for word, weight in severity_keywords.items() if word in text)

df_clean['Violation Severity Score'] = df_clean['Violations'].apply(compute_severity)

# ✅ Limit to top 30 facility types
top_30 = df_clean['Facility Type'].value_counts().nlargest(30).index
df_filtered = df_clean[df_clean['Facility Type'].isin(top_30)]

# ✅ Group and aggregate raw values
df_grouped = (
    df_filtered.groupby('Facility Type')
    .agg({
        'Inspection Score': 'mean',
        'Violation Count': 'mean',
        'Violation Severity Score': 'mean'
    })
    .reset_index()
)

# ✅ Normalize after aggregation
scaler = MinMaxScaler()
df_grouped[['Score_norm', 'Count_norm', 'Severity_norm']] = scaler.fit_transform(
    df_grouped[['Inspection Score', 'Violation Count', 'Violation Severity Score']]
)

# ✅ Save to file
df_grouped.to_csv('data/score_violation_parallel.csv', index=False)
print("✅ Fixed and saved: data/score_violation_parallel.csv")


✅ Fixed and saved: data/score_violation_parallel.csv


In [19]:
df_bubble = (
    df_viz
    .groupby('Facility Type')
    .agg({
        'Violation Count': 'mean',
        'Violation Severity Score': 'mean'
    })
    .reset_index()
    .rename(columns={
        'Violation Count': 'Avg Violation Count',
        'Violation Severity Score': 'Avg Severity Score'
    })
)

# Optional: Keep top 50 largest bubbles
df_bubble = df_bubble.nlargest(50, 'Avg Violation Count')

df_bubble.to_csv('data/packed_bubble_data.csv', index=False)
